# Main Notebook for Prompt Generation, Processing & Visualization

To run the subsequent code, you will need the packages listed below. 

Note that bitsandbytes, accelerate, etc. are only needed if GPU is used. 

In the end, we did not use GPU since computational burden was manageable.

In [1]:
# !pip -q install pandas squarify langchain docx2txt pypdf bitsandbytes accelerate xformers einops datasets loralib sentencepiece sentence_transformers chromadb

In [1]:
import pandas as pd
import io
import re
import requests
import json
import functools 

/Users/jan/opt/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


The data we use consists of a subset of inner joined version of the two Json files 'project-documents.json' and 
'project-dataset.json'.

The specific subset we chose are projects that are associated to one project document either in pdf or docx format.

In [2]:
df = pd.read_csv("processed_data.csv")

We will be accesing the project files whose urls are give in the column named 'url'

In [3]:
df.columns

Index(['project-id', 'recipient-country-details', 'donor-details',
       'project-title', 'sector', 'project-start', 'project-end',
       'project-category', 'project-subcategory', 'total-budget($)',
       'total-expenditure($)', 'DataID', 'ModifyDate', 'FileName', 'category',
       'type', 'id', 'name', 'last_modified', 'file_name', 'url', 'mime_type',
       'description'],
      dtype='object')

In [4]:
import torch
import gc
import os
import docx2txt
from langchain.document_loaders import PyPDFLoader,Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,BitsAndBytesConfig
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain_community.llms import DeepInfra

We set the API token key obtained from DeepInfra. Downloading the llm locally was too computationally expensive so we use their computing cluster instead.

In [5]:
os.environ["DEEPINFRA_API_TOKEN"] = "zAMiCKmaweiQ0nNM0jMaV956QXXjTF1a"

In [6]:
llm = DeepInfra(model_id="meta-llama/Llama-2-70b-chat-hf")
llm.model_kwargs = {
    "temperature": 0,
    "repetition_penalty": 1.2,
    "max_new_tokens": 250,
    "top_p": 0.9,
}

Below is the template for the SDGs label classification task. 

It takes project document as an extra context to answer the classification question (that is we are doing Retrieval Augumented Generation).

A lot of thoghs were put into phrasing the prompt, some are mentioned in the slides.

In [7]:
prompt1_template="""
Based on the context, provide a list of three or less most relevant Sustainable Development Goals (SDGs)
to the project description we supply in the question:

There are 17 Sustainable Development Goals (SDGs) in total, each named as follows:
  1. No poverty
  2. Zero hunger
  3. Good health and well-being
  4. Quality education
  5. Gender equality
  6. Clean water and sanitation
  7. Affordable and clean energy
  8. Decent work and economic growth
  9. Industry, innovation, and infrastructure
  10. Reduced inequalities
  11. Sustainable cities and communities
  12. Responsible consumption and production
  13. Climate action
  14. Life below water
  15. Life on land
  16. Peace, justice, and strong institutions
  17. Partnerships for the goals

Your answer must follow the three rules 1,2 and 3 below:
1. At maximum, the project can be categorized into three SDGs, but usually one or two will suffice.
2. After the list of three or less categories, provide a text of justification (with the header "Justification:") by referring to relevant soureces in the context.
3. If the provided sources in the context were not relevant in the categorization, explicitly mention that these sources played no part.

{context}
Question: {question}
Helpful Answer:
"""

In [8]:
prompt2_template="""
Based on the context, assign a grade on a scale from 1 to 10 to EACH one of the 3 UNIDO priorities. 
The grade assigned must correspond to its relation to the project (1= not connected to the project at all) and (10=strongly connected to the project).

There are 3 UNIDO Priorities in total that you *MUST* assign grades:

1. Sustainable Supply Chains: UNIDO aims to ensure that developing country producers
receive a fair deal while preserving scarce resources. By promoting sustainable supply
chains, it contributes to job creation and equitable economic growth.
 
2. Climate Action: UNIDO addresses climate change by advocating for renewable energy
and energy efficiency in industrial processes. This helps reduce greenhouse gas
emissions and supports a more sustainable future.
 
3. Ending Hunger: UNIDO focuses on cutting post-harvest losses and developing
agribusiness value chains. By enhancing food security and promoting efficient
agricultural practices, it contributes to the goal of ending hunger globally.

Your answer must follow the two rules A,B below:

A. Follow up the answer by providing the justification (with the header "Justification:") by referring to relevant sources in the context.
B. If the provided sources in the context were not relevant in the categorization, explicitly mention that these sources played no part.

{context}
Question: {question}
Helpful Answer:
"""

In [9]:
prompt3_template="""
Based on the context, assign a grade on a scale from 1 to 10 to EACH one of the 5 UNIDO focus areas. 
The grade assigned must correspond to its relation to the project (1= not connected to the project at all) and (10=strongly connected to the project).

There are 5 UNIDO focus areas in total that you *MUST* assign grades to:

1. Digital Transformation and 4IR: involves UNIDO aiding 

member countries in harnessing rapid digital advances to revolutionize manufacturing and 

global value chains. Efforts include smart production applications, fostering 

entrepreneurship, and building industrial skills to boost modernization and MSME 

competitiveness, with a focus on ensuring inclusive participation for women and youth. 
2. Innovation and Technology Absorption: reflects UNIDO's cross-sectoral expertise in promoting 

technology adoption and innovative solutions across diverse areas such as smart energy, 

industrial resilience, and the circular economy. This encompasses technology promotion, 

innovative financing, and developing quality infrastructure to tackle multifaceted 

development challenges and enhance global industrial competitiveness. 

 
3. Circularity, Waste and Pollutants Reduction: UNIDO promotes the transition to circular economy models to minimize waste and pollutants 

across global and domestic value chains. This includes assisting countries in developing 

strategies to comply with environmental agreements and fostering innovation ecosystems 

that catalyze new industries focused on reducing the environmental footprint of industrial 

activities. 


4. Climate Change Mitigation and Adaptation Industry: UNIDO supports the acceleration of renewable energy and energy efficiency initiatives to 

decarbonize industrial activities and power low-carbon, resilient industrialization. It equips 

stakeholders with the knowledge to leverage digital technologies for sustainable industry and 

to develop climate-positive, resilient industrial systems across various sectors.																					                5. Structural transformation and sectoral expertise: is the central pathway for economic development and poverty 

eradication. UNIDO ́s sectoral expertise is grounded on the Organization ́s hands-on support 

to governments and industries of all sizes, including MSMEs. It spans across sectors, from 

agro-food to textile, from automotive to creative, from heavy industry to construction 

material, from medical industry to clean energy, to name a few. 



Your answer must follow the two rules A,B below:

A. Follow up the answer by providing the justification (with the header "Justification:") by referring to relevant sources in the context.
B. If the provided sources in the context were not relevant in the categorization, explicitly mention that these sources played no part.

{context}
Question: {question}
Helpful Answer:
"""

Below is the regex pattern used to extract relevant information from the subsequent classification tasks (UNIDO priority and focus area classification and evaluation tasks). 

Response from the LLM not compatible with these regex patterns are treated as missing data, which is why the performace for these two tasks were not great (but we can improve the regex patterns for better performance had we more time).

In [10]:
strpattern =r"no poverty|"+\
            "zero hunger|"+\
            "good health and well-being|"+\
            "quality education|"+\
            "gender equality|"+\
            "clean water and sanitation|"+\
            "affordable and clean energy|"+\
            "decent work and economic growth|"+\
            "industry, innovation, and infrastructure|"+\
            "reduced inequalities|"+\
            "sustainable cities and communities|"+\
            "responsible consumption and production|"+\
            "climate action|"+\
            "life below water|"+\
            "life on land|"+\
            "peace, justice, and strong institutions|"+\
            "partnerships for the goals"
sdgdic ={"no poverty":1,
    "zero hunger":2,
    "good health and well-being":3,
    "quality education":4,
    "gender equality":5,
    "clean water and sanitation":6,
    "affordable and clean energy":7,
    "decent work and economic growth":8,
    "industry, innovation, and infrastructure":9,
    "reduced inequalities":10,
    "sustainable cities and communities":11,
    "responsible consumption and production":12,
    "climate action":13,
    "life below water":14,
    "life on land":15,
    "peace, justice, and strong institutions":16,
    "partnerships for the goals":17}

In [11]:
strpattern2 =r"digital transformation and 4ir[:\d\s]+|"+\
            "innovation and technology absorption[:\d\s]+|"+\
            "circularity, waste and pollutants reduction[:\d\s]+|"+\
            "climate change mitigation and adaptation industry[:\d\s]+|"+\
            "structural transformation and sectoral expertise[:\d\s]+"

The main function to carry out the RAG which is basically asking an LLM question while instructing it to answer by referring to relevant sources.

In [12]:
def RAG_from_prompt(question,prompt_tmp,db):
    QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt_tmp)
    llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, callbacks=None, verbose=False)
    # combine documents retrieved from vector database
    document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
    )
    combine_documents_chain = StuffDocumentsChain(
        llm_chain=llm_chain,
        document_variable_name="context",
        document_prompt=document_prompt,
        callbacks=None,
    )
    # feed all above into this function to carry out the QA
    qa_chain = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    callbacks=None,
    verbose=False,
    retriever = db.as_retriever(search_kwargs={'k':5}),
    return_source_documents = True
    )
    return(qa_chain(question))

The for loop below will run through the entire document if you comment out the "if(urls_idx == 10): break" line

In [14]:
document_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n","\n"," ", ""],
                                                   chunk_size = 1000,
                                                   chunk_overlap = 200,
                                                   length_function = len)
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')  
Outputs_SDG = []
Outputs_Priorities = []
Outputs_FocusAreas = []
for url_idx,url in enumerate(df['url']):
    print(url_idx)
    if(url_idx == 10):
        break
    filename = re.split(r'/',url)[-1]
    with open("proj_pdf_tmp/"+filename,'wb') as f:
        r = requests.get(url)
        f.write(r.content)
    document = []
    ext = re.split(r"\.",url)[-1]
    if ext == 'pdf':
        loader = PyPDFLoader("proj_pdf_tmp/" + filename)
    elif ext == 'docx':
        loader = Docx2txtLoader("proj_pdf_tmp/" + filename)
    else:
        raise ValueError("Only pdf and docx files are read")
    document.extend(loader.load())
    document_chunks = document_splitter.split_documents(document)
    if len(document_chunks)==0:
        continue
    vectordb = Chroma.from_documents(document_chunks, embedding = embeddings)    
    # Extract country name from the recipient-***-details which is in json format
    data_string = df.iloc[0].loc['recipient-country-details']
    data_string = data_string.replace("'", '"')
    data_list = json.loads(data_string)
    country_name = data_list[0]['recipient-country']
    question = "This is a United Nations Industrial Development Organization (UNIDO) project called \"" +\
    df.iloc[url_idx]['project-title'] +\
    "\" taking place in " + country_name +\
    ". This project is internally categorized as " +\
    df.iloc[url_idx]['project-category'] +" with a subcategory " +\
    df.iloc[url_idx]['project-subcategory']
    
    # SDGs classification
    result1 = RAG_from_prompt(question, prompt1_template, vectordb)
    categorization = re.split(r'Justification:',result1['result'])[0].lower()
    try :
        justification = re.split(r'Justification:',result1['result'])[1].lower()
    except IndexError:
        justification = "No justification or the llm did not comply with the prompt instruction"          
    SDGs_txt = set(re.findall(strpattern,categorization))
    SDGs_number = sorted(list(map(lambda x: sdgdic[x],SDGs_txt)))
    Outputs_SDG.append([df.iloc[url_idx]['project-id'],SDGs_number,justification,result1['source_documents']])
    
    # Priorities classification
    result2 = RAG_from_prompt(question, prompt2_template, vectordb)
    try :
        justification2 = re.split(r'Justification:',result2['result'])[1].lower()
    except IndexError:
        justification2 = "No justification or the llm did not comply with the prompt instruction" 
    Priorities_scores_tmp = re.findall(r'ending hunger[:\d\s]+|climate action[:\d\s]+|sustainable supply chains[:\d\s]+',
               result2['result'].lower())
    
    try:
        Priorities_scores = {sublist[0]: sublist[1] for sublist in list(map(lambda z: [functools.reduce(lambda x,y: x+' '+y,re.findall(r'[a-z]+',z)),
                  int(re.findall(r'\d+',z)[0])], Priorities_scores_tmp))}
    except IndexError:
        Priorities_scores = {}
    Outputs_Priorities.append([df.iloc[url_idx]['project-id'],Priorities_scores,justification2,result2['source_documents']])
    
    # Focus Areas
    result3 = RAG_from_prompt(question, prompt3_template, vectordb)
    try :
        justification3 = re.split(r'Justification:',result3['result'])[1].lower()
    except IndexError:
        justification3 = "No justification or the llm did not comply with the prompt instruction" 
    FA_scores_tmp = re.findall(strpattern2,result3['result'].lower())
    try:
        FA_scores = {sublist[0]: sublist[1] for sublist in list(map(lambda z: [functools.reduce(lambda x,y: x+' '+y,re.findall(r'[a-z]+',z)),
                  int(re.findall(r'\d+$',z)[0])], FA_scores_tmp))}
    except IndexError:
        FA_scores ={}
    Outputs_FocusAreas.append([df.iloc[url_idx]['project-id'],FA_scores,justification3,result3['source_documents']])

/Users/jan/opt/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


0
1
2
3
4
5
6
7
8
9
10


Below are data frames we obtain from feeding the prompts to the LLM for each project

In [15]:
# SDGs task
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
display(pd.DataFrame(Outputs_SDG, columns=['project-id','SDG-labels','Justification','Source']))


,project-id,SDG-labels,Justification,Source
0,180267,"[7, 12, 14, 15]",No justification or the llm did not comply wit...,[page_content='mechanisms to handle hazardous ...
1,210120,"[6, 7, 13]","\naccording to annex b of the first source, th...",[page_content='12 \n \n \nAnnex B: Project e...
2,120221,"[12, 13, 15]",\naccording to annex b of the project document...,[page_content='12 \n \n \nAnnex B: Project e...
3,180268,"[8, 12, 15]",\nthe project focuses on improving working con...,[page_content='point for change. Environmental...
4,190025,"[7, 13, 17]",\naccording to the project's title and objecti...,[page_content='12 \n \n \nAnnex B: Project e...
5,230007,"[9, 12, 17]","\nthis project focuses on promoting industry, ...",[page_content='at UNIDO to organize the Global...
6,200174,"[12, 13, 15]","\naccording to the first source, the project's...",[page_content='12 \n \n \nAnnex B: Project e...
7,190344,"[1, 8, 11, 13]","\nfrom the provided context, it is clear that ...",[page_content='may be transferred to national ...
8,190290,"[6, 8, 17]","\naccording to the project document, the goal ...","[page_content=""Produced on 18.10.2019 at ,15:..."


In [23]:
pd.DataFrame.from_dict(Outputs_Priorities[4][1],orient='index',columns=['score'])a

,score
sustainable supply chains,8
climate action,9
ending hunger,4


In [29]:
pd.DataFrame.from_dict(Outputs_FocusAreas[5][1],orient='index',columns=['score'])

,score
digital transformation and ir,8
innovation and technology absorption,9
circularity waste and pollutants reduction,6
climate change mitigation and adaptation industry,7
structural transformation and sectoral expertise,5


In [16]:
# UNIDO Prority task
pd.DataFrame(Outputs_Priorities, columns=['project-id','UNIDO priority evaluation','Justification','Source'])

,project-id,UNIDO priority evaluation,Justification,Source
0,180267,"{'sustainable supply chains': 4, 'climate acti...",paragraph 6 of the second source mentions tha...,[page_content='mechanisms to handle hazardous ...
1,210120,{},no specific mentions of supply chains or mate...,[page_content='12 \n \n \nAnnex B: Project e...
2,120221,{},page 4 of the pdf mentions hcfc-22 and its ha...,[page_content='12 \n \n \nAnnex B: Project e...
3,180268,{},although the project does not directly addres...,[page_content='point for change. Environmental...
4,190025,"{'sustainable supply chains': 8, 'climate acti...","\naccording to the first source, the project a...",[page_content='12 \n \n \nAnnex B: Project e...
5,230007,{},No justification or the llm did not comply wit...,[page_content='at UNIDO to organize the Global...
6,200174,{},No justification or the llm did not comply wit...,[page_content='12 \n \n \nAnnex B: Project e...
7,190344,{},No justification or the llm did not comply wit...,[page_content='may be transferred to national ...
8,190290,{},although the project does not directly addres...,"[page_content=""Produced on 18.10.2019 at ,15:..."


In [24]:
# UNIDO Focus Areas task
pd.DataFrame(Outputs_FocusAreas, columns=['project-id','UNIDO focus area evaluation','Justification','Source'])

,project-id,UNIDO focus area evaluation,Justification,Source
0,180267,{},not directly addressed in the project documen...,[page_content='mechanisms to handle hazardous ...
1,210120,{},No justification or the llm did not comply wit...,[page_content='12 \n \n \nAnnex B: Project e...
2,120221,"{'digital transformation and ir': 6, 'innovati...",no specific mentions in the given context abo...,[page_content='12 \n \n \nAnnex B: Project e...
3,180268,{},No justification or the llm did not comply wit...,[page_content='point for change. Environmental...
4,190025,{},"according to the first source, the project in...",[page_content='12 \n \n \nAnnex B: Project e...
5,230007,"{'digital transformation and ir': 8, 'innovati...",No justification or the llm did not comply wit...,[page_content='at UNIDO to organize the Global...
6,200174,"{'digital transformation and ir': 6, 'innovati...",No justification or the llm did not comply wit...,[page_content='12 \n \n \nAnnex B: Project e...
7,190344,{},No justification or the llm did not comply wit...,[page_content='may be transferred to national ...
8,190290,{},No justification or the llm did not comply wit...,"[page_content=""Produced on 18.10.2019 at ,15:..."


Some visualizations will follow

In [18]:
lands = []
for idx,val in enumerate(df['recipient-country-details']):
    try: 
        lands.append(json.loads(val.replace("'",'"'))[0]['recipient-country'])
    except ValueError:
        lands.append("Unknown")

In [19]:
import matplotlib.pyplot as plt
import squarify
from collections import Counter

def plot_treemap_from_vector(vector):
    frequency_dict = Counter(vector)

    labels = list(frequency_dict.keys())
    sizes = list(frequency_dict.values())

    plt.figure(figsize=(10, 6))
    squarify.plot(sizes=sizes, label=labels, pad=True)
    plt.title("Treemap of SDG labels for Global Projects")
    plt.axis('off')
    #plt.savefig('foo.png', bbox_inches='tight')
    plt.show()

df = pd.DataFrame(Outputs_SDG, columns=['project-id','SDG-labels','Justification','Source'])
# Example vector of integers
SDGs = functools.reduce(lambda x,y: x+y ,df[pd.Series(lands)=="Global"]["SDG-labels"])

plot_treemap_from_vector(list(map(lambda x: "SDG " + str(x),SDGs)))

/var/folders/zn/z5kr_syd26x_fkh2s75pqgdc0000gn/T/ipykernel_3934/3230619238.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  SDGs = functools.reduce(lambda x,y: x+y ,df[pd.Series(lands)=="Global"]["SDG-labels"])


TypeError: reduce() of empty iterable with no initial value

In [ ]:
import matplotlib.pyplot as plt
yvals = list(map(lambda x: len(x), df['SDG-labels']))
xvals =df['project-id']
plt.yticks(range(0,18))
plt.plot(yvals)
plt.xlabel('Data Frame Row Index')
plt.ylabel('Number of SDG labels')
plt.title('Number of SDG Labels Assigned to Projects')
plt.savefig('foo3.png', bbox_inches='tight')